In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

#from tensorflow.keras.preprocessing import image_dataset_from_directory
print(tf.__version__)

2.0.0


In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

!wget --no-check-certificate \
    https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip
  
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()
# Directory with our training horse pictures
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

# Directory with our training horse pictures
validation_horse_dir = os.path.join('/tmp/validation-horse-or-human/horses')

# Directory with our training human pictures
validation_human_dir = os.path.join('/tmp/validation-horse-or-human/humans')

--2020-08-22 22:30:04--  https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.128, 74.125.20.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M   190MB/s    in 0.8s    

2020-08-22 22:30:04 (190 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]

--2020-08-22 22:30:05--  https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M)

**Model 2**

In [3]:
BATCH_SIZE = 32
IMG_SIZE = (300, 300)

In [4]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [5]:
# Make all the layers in the pre-trained model non-trainable
for layer in base_model.layers:
  layer.trainable = False

In [6]:
last_layer = base_model.get_layer('out_relu')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 10, 10, 1280)


In [7]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')

In [8]:
x = global_average_layer(last_output)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)

In [9]:
model = tf.keras.Model(base_model.input, outputs)

In [10]:
from tensorflow.keras.optimizers import RMSprop
model.compile(
    optimizer=RMSprop(lr=0.0001), 
    loss='binary_crossentropy', 
    metrics=['accuracy']
)


In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/tmp/horse-or-human/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        '/tmp/validation-horse-or-human/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [13]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch= 1027/ 32,
    epochs=10,
    validation_data=validation_generator,
    validation_steps= 256/ 32
)

Epoch 1/10
33/32 [==============================] - 563s 17s/step - loss: 0.6909 - accuracy: 0.5891 - val_loss: 0.6391 - val_accuracy: 0.6016
Epoch 2/10
33/32 [==============================] - 558s 17s/step - loss: 0.6015 - accuracy: 0.6962 - val_loss: 0.5841 - val_accuracy: 0.7148
Epoch 3/10
33/32 [==============================] - 553s 17s/step - loss: 0.5509 - accuracy: 0.7595 - val_loss: 0.5384 - val_accuracy: 0.8242
Epoch 4/10
33/32 [==============================] - 550s 17s/step - loss: 0.4974 - accuracy: 0.8462 - val_loss: 0.4976 - val_accuracy: 0.8789
Epoch 5/10
33/32 [==============================] - 549s 17s/step - loss: 0.4897 - accuracy: 0.8510 - val_loss: 0.4563 - val_accuracy: 0.9297
Epoch 6/10
33/32 [==============================] - 548s 17s/step - loss: 0.4386 - accuracy: 0.8890 - val_loss: 0.4251 - val_accuracy: 0.9219
Epoch 7/10
33/32 [==============================] - 549s 17s/step - loss: 0.4002 - accuracy: 0.9143 - val_loss: 0.3951 - val_accuracy: 0.9141
Epoch 

KeyboardInterrupt: ignored

In [14]:
model.save("category3.h5")

In [ ]:
import os
print(str(os.path.getsize('./category3.h5')/1000000) + 'MB')

9.517792MB


In [17]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [18]:
!pip install tensorflow==2.0.0
import tensorflow as tf
print(tf.__version__)

     |████████████████████████████████| 86.3MB 117kB/s 
     |████████████████████████████████| 3.8MB 50.9MB/s 
     |████████████████████████████████| 450kB 44.3MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=00678df1cfe16ffa4f43b9f5befac31f86c5011de2815db97fe7367013861b6e
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tens

2.3.0
